# Chapter 6 -- Machine Learning with scikit-learn
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/CH06_Machine_Learning_Sklearn.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Part:** 3 -- Machine Learning and AI  
**Prerequisites:** Chapter 5 (SciPy and Statistical Computing)  
**Estimated time:** 6-7 hours

---

### Learning Objectives

By the end of this chapter you will be able to:

- Explain the supervised vs unsupervised learning distinction
- Build a complete scikit-learn preprocessing pipeline with `Pipeline` and `ColumnTransformer`
- Train and evaluate regression models: Linear, Ridge, Lasso, Random Forest
- Train and evaluate classification models: Logistic Regression, Decision Tree, Random Forest, Gradient Boosting
- Use cross-validation correctly and interpret learning curves
- Apply unsupervised learning: KMeans clustering and PCA dimensionality reduction
- Interpret feature importances and SHAP-style analysis
- Tune hyperparameters with GridSearchCV and RandomizedSearchCV

---

### Project Thread -- Chapter 6

Three complete ML tasks on the SO 2025 dataset:

1. **Regression** -- predict annual salary from developer profile features
2. **Classification** -- predict whether a developer uses Python as their primary language
3. **Clustering** -- discover natural groupings of developers by skills and compensation

Each task follows the same professional workflow: clean data, build pipeline,
train, cross-validate, evaluate, and interpret.


---

## Setup -- Imports and Data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import (
    train_test_split, cross_val_score, KFold,
    GridSearchCV, RandomizedSearchCV, learning_curve
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
)
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, classification_report, confusion_matrix,
    silhouette_score
)

import sklearn
print(f'scikit-learn: {sklearn.__version__}')
print(f'NumPy:        {np.__version__}')
print(f'Pandas:       {pd.__version__}')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi']       = 110
plt.rcParams['axes.titlesize']   = 13
plt.rcParams['axes.titleweight'] = 'bold'

DATASET_URL = 'https://raw.githubusercontent.com/timothy-watt/python-for-ai-ml/main/data/so_survey_2025_curated.csv'
RANDOM_STATE = 42


In [ ]:
# Load and clean SO 2025 -- full pipeline from Chapter 3
df_raw = pd.read_csv(DATASET_URL)
df = df_raw.copy()

# Salary target
df = df.dropna(subset=['ConvertedCompYearly'])
df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')
Q1, Q3 = df['ConvertedCompYearly'].quantile([0.25, 0.75])
IQR = Q3 - Q1
df = df[
    (df['ConvertedCompYearly'] >= max(Q1 - 3*IQR, 5_000)) &
    (df['ConvertedCompYearly'] <= min(Q3 + 3*IQR, 600_000))
].copy()

# Numeric features
if 'YearsCodePro' in df.columns:
    df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# Categorical features
for col in ['Country', 'EdLevel', 'Employment', 'RemoteWork', 'OrgSize', 'DevType']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# Derived features
df['uses_python'] = df.get('LanguageHaveWorkedWith', pd.Series(dtype=str)).str.contains('Python', na=False).astype(int)
df['uses_sql']    = df.get('LanguageHaveWorkedWith', pd.Series(dtype=str)).str.contains('SQL', na=False).astype(int)
df['uses_js']     = df.get('LanguageHaveWorkedWith', pd.Series(dtype=str)).str.contains('JavaScript', na=False).astype(int)
df['uses_ai']     = df.get('AIToolCurrently', pd.Series(dtype=str)).notna().astype(int)
df['log_salary']  = np.log(df['ConvertedCompYearly'])

df = df.reset_index(drop=True)
print(f'Dataset ready: {len(df):,} rows x {df.shape[1]} columns')
print(f'Target (salary): ${df["ConvertedCompYearly"].median():,.0f} median')


---

## Section 6.1 -- The Machine Learning Framework

### Supervised vs Unsupervised Learning

**Supervised learning** trains on labelled examples -- pairs of (input features, known target).
The model learns to map inputs to outputs. Two subtypes:
- *Regression*: target is a continuous number (salary, house price, temperature)
- *Classification*: target is a discrete category (Python user / not, spam / not spam)

**Unsupervised learning** finds structure in data without labels.
- *Clustering*: group similar items together (developer archetypes)
- *Dimensionality reduction*: compress many features into fewer (PCA)

### The scikit-learn API Contract

Every scikit-learn estimator follows the same three-method contract:
```python
model.fit(X_train, y_train)       # learn from training data
model.predict(X_test)             # apply to new data
model.score(X_test, y_test)       # evaluate performance
```
Every transformer adds:
```python
transformer.transform(X)          # apply the learned transformation
transformer.fit_transform(X)      # fit and transform in one step
```
This consistent API is why switching between models is a one-line change.


In [ ]:
# 6.1.1 -- The train/test split: the most important rule in ML
#
# A model evaluated on the data it trained on will ALWAYS look better than it is.
# It has simply memorised the training data -- a phenomenon called overfitting.
# The test set must NEVER be seen during training or hyperparameter tuning.

# Define features and target for the regression task
feature_cols = [c for c in ['YearsCodePro', 'uses_python', 'uses_sql', 'uses_js', 'uses_ai']
                if c in df.columns]
target_col   = 'log_salary'   # we predict log(salary) and exponentiate back

X = df[feature_cols].fillna(df[feature_cols].median())
y = df[target_col]

# 80/20 train-test split, stratified by salary quartile to preserve distribution
sal_quartile = pd.qcut(y, q=4, labels=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=sal_quartile
)

print(f'Training set:  {len(X_train):,} rows  ({len(X_train)/len(X)*100:.0f}%)')
print(f'Test set:      {len(X_test):,} rows  ({len(X_test)/len(X)*100:.0f}%)')
print(f'Features:      {feature_cols}')
print(f'Target:        log_salary (exponentiate predictions to get USD)')

# Verify distribution is preserved
print(f'Train median log-salary: {y_train.median():.4f}')
print(f'Test  median log-salary: {y_test.median():.4f}')


---

## Section 6.2 -- Regression: Predicting Salary

We build a complete regression pipeline with preprocessing and compare
four models: Linear Regression, Ridge, Lasso, and Random Forest.
All models are evaluated with 5-fold cross-validation on the training set
before a single look at the test set.


In [ ]:
# 6.2.1 -- Building a preprocessing Pipeline
#
# The Pipeline chains preprocessing steps and the model in one object.
# This guarantees that:
#   1. Scalers are fit ONLY on training data (no data leakage)
#   2. The same transformations are applied identically to test data
#   3. The entire workflow is serialisable -- save one object, get everything

# Numeric pipeline: impute missing values with median, then standardise
numeric_features = [c for c in ['YearsCodePro', 'uses_python', 'uses_sql', 'uses_js', 'uses_ai']
                    if c in df.columns]

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),   # fill NaN with column median
    ('scaler',  StandardScaler()),                    # zero mean, unit variance
])

# ColumnTransformer applies different pipelines to different column groups
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
], remainder='drop')   # drop any columns not explicitly listed

# Full pipelines: preprocessor + model in one object
pipelines = {
    'Linear Regression': Pipeline([
        ('prep',  preprocessor),
        ('model', LinearRegression())
    ]),
    'Ridge (L2)': Pipeline([
        ('prep',  preprocessor),
        ('model', Ridge(alpha=1.0))
    ]),
    'Lasso (L1)': Pipeline([
        ('prep',  preprocessor),
        ('model', Lasso(alpha=0.01))
    ]),
    'Random Forest': Pipeline([
        ('prep',  preprocessor),
        ('model', RandomForestRegressor(
            n_estimators=100,
            max_depth=8,
            random_state=RANDOM_STATE,
            n_jobs=-1   # use all CPU cores
        ))
    ]),
}

print('Pipelines built:')
for name in pipelines:
    print(f'  {name}')


In [ ]:
# 6.2.2 -- Cross-validation: honest model evaluation
#
# 5-fold CV splits the training data into 5 parts, trains on 4, validates on 1,
# and rotates until every fold has been the validation set once.
# This gives 5 independent performance estimates -- their mean and std
# are much more trustworthy than a single train/validate split.

kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

cv_results = {}
print(f'5-fold cross-validation on training set (n={len(X_train):,})...')
print(f'{"Model":<22} {"CV R^2 mean":>12} {"CV R^2 std":>12} {"CV MAE mean":>14}')
print('-' * 62)

for name, pipe in pipelines.items():
    r2_scores  = cross_val_score(pipe, X_train, y_train, cv=kfold, scoring='r2')
    mae_scores = cross_val_score(pipe, X_train, y_train, cv=kfold,
                                 scoring='neg_mean_absolute_error')
    cv_results[name] = {
        'r2_mean':  r2_scores.mean(),
        'r2_std':   r2_scores.std(),
        'mae_mean': -mae_scores.mean(),
    }
    print(f'{name:<22} {r2_scores.mean():>12.4f} {r2_scores.std():>12.4f} {-mae_scores.mean():>14.4f}')

best_model_name = max(cv_results, key=lambda k: cv_results[k]['r2_mean'])
print(f'Best model by CV R^2: {best_model_name}')


In [ ]:
# 6.2.3 -- Final evaluation on the held-out test set
#
# We do this ONCE -- after all model selection is done using CV.
# The test set result is our honest estimate of real-world performance.

best_pipe = pipelines[best_model_name]
best_pipe.fit(X_train, y_train)

y_pred_log  = best_pipe.predict(X_test)
y_pred_usd  = np.exp(y_pred_log)     # convert log predictions back to USD
y_true_usd  = np.exp(y_test)         # convert log actuals back to USD

mae  = mean_absolute_error(y_true_usd, y_pred_usd)
rmse = np.sqrt(mean_squared_error(y_true_usd, y_pred_usd))
r2   = r2_score(y_test, y_pred_log)  # R^2 on log scale (more meaningful)

print(f'Test set results ({best_model_name}):')
print(f'  R^2 (log scale):  {r2:.4f}')
print(f'  MAE  (USD):       ${mae:,.0f}')
print(f'  RMSE (USD):       ${rmse:,.0f}')

# Residual plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(y_true_usd/1000, y_pred_usd/1000, alpha=0.25, s=10, color='#2E75B6')
lim = max(y_true_usd.max(), y_pred_usd.max()) / 1000
axes[0].plot([0, lim], [0, lim], 'r--', linewidth=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Salary ($k)')
axes[0].set_ylabel('Predicted Salary ($k)')
axes[0].set_title(f'{best_model_name}\nActual vs Predicted Salary')
axes[0].legend()

residuals = y_true_usd - y_pred_usd
axes[1].scatter(y_pred_usd/1000, residuals/1000, alpha=0.25, s=10, color='#E8722A')
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Salary ($k)')
axes[1].set_ylabel('Residual ($k)')
axes[1].set_title('Residual Plot\n(random scatter = good; pattern = model problem)')

plt.tight_layout()
plt.show()


In [ ]:
# 6.2.4 -- Feature importance (Random Forest)

# Always use the regression RF pipeline -- fit it on regression training data
rf_pipe = pipelines['Random Forest']
rf_pipe.fit(X_train, y_train)
rf_model    = rf_pipe.named_steps['model']
importances = rf_model.feature_importances_

# Get feature names from the pipeline's preprocessor (handles any column ordering)
try:
    feat_names = rf_pipe.named_steps['prep'].get_feature_names_out()
    feat_names = [n.split('__')[-1] for n in feat_names]   # strip 'num__' prefix
except Exception:
    feat_names = numeric_features   # fallback to original list

feat_imp = pd.Series(importances, index=feat_names[:len(importances)]).sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(9, 4))
colors = ['#E8722A' if i == feat_imp.index[-1] else '#2E75B6' for i in feat_imp.index]
feat_imp.plot(kind='barh', ax=ax, color=colors)
ax.set_title('Random Forest: Feature Importances for Salary Prediction')
ax.set_xlabel('Importance (mean decrease in impurity)')
for i, v in enumerate(feat_imp.values):
    ax.text(v + 0.002, i, f'{v:.3f}', va='center', fontsize=9)
plt.tight_layout()
plt.show()

print('Feature importances (higher = more predictive of salary):')
for feat, imp in feat_imp.sort_values(ascending=False).items():
    print(f'  {feat:<25} {imp:.4f}')


---

## Section 6.3 -- Classification: Predicting Python Usage

We now build a classification pipeline that predicts whether a developer
uses Python as their primary language, based on their salary, experience,
and other profile features. This demonstrates the full classification workflow:
encoding, class balance, multiple metrics, and confusion matrix interpretation.


In [ ]:
# 6.3.1 -- Build classification features and handle categorical encoding

# Target: uses_python (already a 0/1 column)
clf_target = 'uses_python'

# Use numeric + one-hot encoded categorical features
num_cols = [c for c in ['YearsCodePro', 'ConvertedCompYearly', 'uses_sql', 'uses_js', 'uses_ai']
            if c in df.columns]
cat_cols = [c for c in ['EdLevel', 'RemoteWork'] if c in df.columns]

# Build a ColumnTransformer that handles both types
clf_preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler',  StandardScaler()),
    ]), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
        ('ohe',     OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ]), cat_cols),
], remainder='drop')

X_clf = df[num_cols + cat_cols]
y_clf = df[clf_target]

X_tr, X_te, y_tr, y_te = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=RANDOM_STATE, stratify=y_clf
)

print(f'Classification target: {clf_target}')
print(f'Class balance:  Python={y_clf.mean()*100:.1f}%,  Non-Python={100-y_clf.mean()*100:.1f}%')
print(f'Train: {len(X_tr):,}   Test: {len(X_te):,}')


In [ ]:
# 6.3.2 -- Train and cross-validate four classifiers

clf_pipelines = {
    'Logistic Regression': Pipeline([
        ('prep',  clf_preprocessor),
        ('model', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
    ]),
    'Decision Tree': Pipeline([
        ('prep',  clf_preprocessor),
        ('model', DecisionTreeClassifier(max_depth=6, random_state=RANDOM_STATE))
    ]),
    'Random Forest': Pipeline([
        ('prep',  clf_preprocessor),
        ('model', RandomForestClassifier(
            n_estimators=100, max_depth=8,
            random_state=RANDOM_STATE, n_jobs=-1
        ))
    ]),
    'Gradient Boosting': Pipeline([
        ('prep',  clf_preprocessor),
        ('model', GradientBoostingClassifier(
            n_estimators=100, max_depth=4,
            random_state=RANDOM_STATE
        ))
    ]),
}

print(f'5-fold CV on training set (n={len(X_tr):,})...')
print(f'{"Model":<22} {"Accuracy":>10} {"Std":>8}')
print('-' * 42)

clf_cv = {}
kfold  = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for name, pipe in clf_pipelines.items():
    scores = cross_val_score(pipe, X_tr, y_tr, cv=kfold, scoring='accuracy')
    clf_cv[name] = scores
    print(f'{name:<22} {scores.mean():>10.4f} {scores.std():>8.4f}')

best_clf_name = max(clf_cv, key=lambda k: clf_cv[k].mean())
print(f'Best classifier by CV accuracy: {best_clf_name}')


In [ ]:
# 6.3.3 -- Evaluate best classifier on the test set

best_clf = clf_pipelines[best_clf_name]
best_clf.fit(X_tr, y_tr)
y_pred_clf = best_clf.predict(X_te)

acc = accuracy_score(y_te, y_pred_clf)
print(f'Test accuracy ({best_clf_name}): {acc:.4f}  ({acc*100:.1f}%)')
print()
print('Classification Report:')
print(classification_report(y_te, y_pred_clf,
                             target_names=['Non-Python', 'Python']))

# Confusion matrix
cm = confusion_matrix(y_te, y_pred_clf)
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Non-Python', 'Python'],
    yticklabels=['Non-Python', 'Python'],
    ax=ax
)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title(f'{best_clf_name}\nConfusion Matrix')
plt.tight_layout()
plt.show()

# Interpret the confusion matrix
tn, fp, fn, tp = cm.ravel()
print(f'True Negatives  (correctly predicted Non-Python): {tn:,}')
print(f'False Positives (predicted Python, actually not): {fp:,}')
print(f'False Negatives (predicted Non-Python, actually Python): {fn:,}')
print(f'True Positives  (correctly predicted Python): {tp:,}')


In [ ]:
# 6.3.4 -- Learning curves: diagnosing bias vs variance
#
# A learning curve shows how training and validation score change
# as we add more training data.
#
# High bias (underfitting):  both curves plateau low
# High variance (overfitting): training score >> validation score
# Good fit: both curves converge to a high value

train_sizes, train_scores, val_scores = learning_curve(
    best_clf, X_tr, y_tr,
    cv=5, scoring='accuracy',
    train_sizes=np.linspace(0.1, 1.0, 8),
    n_jobs=-1
)

train_mean = train_scores.mean(axis=1)
train_std  = train_scores.std(axis=1)
val_mean   = val_scores.mean(axis=1)
val_std    = val_scores.std(axis=1)

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(train_sizes, train_mean, 'o-', color='#E8722A', linewidth=2, label='Training score')
ax.fill_between(train_sizes, train_mean-train_std, train_mean+train_std,
                alpha=0.15, color='#E8722A')
ax.plot(train_sizes, val_mean, 'o-', color='#2E75B6', linewidth=2, label='Validation score')
ax.fill_between(train_sizes, val_mean-val_std, val_mean+val_std,
                alpha=0.15, color='#2E75B6')
ax.set_xlabel('Training set size')
ax.set_ylabel('Accuracy')
ax.set_title(f'Learning Curve: {best_clf_name}')
ax.legend(fontsize=10)
ax.set_ylim(0.5, 1.01)
plt.tight_layout()
plt.show()

gap = train_mean[-1] - val_mean[-1]
print(f'Final training accuracy:   {train_mean[-1]:.4f}')
print(f'Final validation accuracy: {val_mean[-1]:.4f}')
print(f'Gap (train - val):         {gap:.4f}  ', end='')
if gap < 0.02:   print('(excellent fit -- low variance)')
elif gap < 0.08: print('(mild overfitting -- acceptable)')
else:            print('(overfitting -- reduce model complexity or add regularisation)')


---

## Section 6.4 -- Hyperparameter Tuning

Hyperparameters are settings you choose before training -- things like
tree depth, regularisation strength, or number of estimators.
They are not learned from data; you have to search for good values.

Two strategies:
- **GridSearchCV**: exhaustively tries every combination in a grid
- **RandomizedSearchCV**: randomly samples combinations -- better when the grid is large

Both use cross-validation internally so the search never touches the test set.


In [ ]:
# 6.4.1 -- RandomizedSearchCV to tune the Random Forest classifier

from scipy.stats import randint, uniform

# Define the parameter distribution to sample from
# Note the naming convention: 'step_name__parameter_name'
param_dist = {
    'model__n_estimators':  randint(50, 300),
    'model__max_depth':     randint(3, 15),
    'model__min_samples_split': randint(2, 20),
    'model__max_features':  ['sqrt', 'log2', 0.5],
}

rf_clf_pipe = Pipeline([
    ('prep',  clf_preprocessor),
    ('model', RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)),
])

search = RandomizedSearchCV(
    rf_clf_pipe,
    param_distributions=param_dist,
    n_iter=20,              # try 20 random combinations
    cv=3,                   # 3-fold CV (faster than 5 during search)
    scoring='accuracy',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)

print('Running RandomizedSearchCV (20 iterations x 3 folds = 60 fits)...')
search.fit(X_tr, y_tr)

print(f'Best CV accuracy: {search.best_score_:.4f}')
print('Best parameters:')
for param, val in search.best_params_.items():
    print(f'  {param}: {val}')

# Evaluate the tuned model on the test set
y_pred_tuned = search.best_estimator_.predict(X_te)
tuned_acc    = accuracy_score(y_te, y_pred_tuned)
print(f'Tuned model test accuracy: {tuned_acc:.4f}  (vs baseline {acc:.4f})')


---

## Section 6.5 -- Unsupervised Learning: Clustering and PCA

Clustering discovers natural groupings in data without labels.
We apply KMeans to the SO 2025 dataset to find archetypes of developers --
groups of respondents who are similar in skills, experience, and compensation.

PCA (Principal Component Analysis) reduces dimensionality for visualisation
and often improves clustering by removing noise from irrelevant features.


In [ ]:
# 6.5.1 -- KMeans clustering: finding developer archetypes

# Build cluster features directly from df using only columns confirmed to exist
# and filled so there are no NaN rows to drop
all_wanted = ['YearsCodePro', 'ConvertedCompYearly',
              'uses_python', 'uses_sql', 'uses_js', 'uses_ai']
cluster_cols = [c for c in all_wanted if c in df.columns]
print(f"Using cluster columns: {cluster_cols}")

X_cluster = df[cluster_cols].copy()

# Fill every column with its own median; fall back to 0 if median is NaN
for col in cluster_cols:
    med = X_cluster[col].median()
    fill_val = med if pd.notna(med) else 0
    X_cluster[col] = X_cluster[col].fillna(fill_val)
# Final safety net: catch any remaining NaNs
X_cluster = X_cluster.fillna(0)

# Confirm no NaNs remain
assert X_cluster.isnull().sum().sum() == 0, "NaNs still present after fill"
X_cluster = X_cluster.reset_index(drop=True)
print(f"Rows for clustering: {len(X_cluster):,}")

# Scale before clustering -- KMeans uses Euclidean distance
# Unscaled features with different ranges distort distances
scaler   = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)
print(f"X_scaled shape: {X_scaled.shape}")

# Choose k with the elbow method
inertias   = []
sil_scores = []
k_range    = range(2, 10)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)
    sil_scores.append(silhouette_score(X_scaled, km.labels_, sample_size=2000))

fig, axes = plt.subplots(1, 2, figsize=(13, 4))
axes[0].plot(k_range, inertias, 'o-', color='#2E75B6', linewidth=2)
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method: choose k where inertia curve bends')
axes[1].plot(k_range, sil_scores, 'o-', color='#E8722A', linewidth=2)
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score: higher is better')
plt.tight_layout()
plt.show()

best_k = list(k_range)[sil_scores.index(max(sil_scores))]
print(f'Best k by silhouette score: {best_k}')


In [ ]:
# 6.5.2 -- Fit final KMeans and profile each cluster

best_k = max(3, best_k)   # ensure at least 3 clusters for interesting profiles

km_final = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
labels   = km_final.fit_predict(X_scaled)

# reset_index ensures the integer positions of labels align with DataFrame rows
X_cluster_labelled = X_cluster.reset_index(drop=True).copy()
X_cluster_labelled['cluster'] = labels

print(f'Cluster profiles (k={best_k}):')
profile = X_cluster_labelled.groupby('cluster')[cluster_cols].mean().round(2)
profile['count'] = X_cluster_labelled.groupby('cluster').size()
print(profile.to_string())

# Name the clusters based on their profiles
print()
print('Cluster interpretation:')
sal_col = 'ConvertedCompYearly'
exp_col = 'YearsCodePro'
if sal_col in profile.columns and exp_col in profile.columns:
    for cluster_id in range(best_k):
        row = profile.loc[cluster_id]
        sal = row[sal_col]
        exp = row[exp_col] if exp_col in row.index else 0
        py  = row['uses_python'] if 'uses_python' in row.index else 0
        ai  = row['uses_ai'] if 'uses_ai' in row.index else 0
        print(f'  Cluster {cluster_id}: ${sal:,.0f} median salary, '
              f'{exp:.0f} yrs exp, Python={py:.0%}, AI tools={ai:.0%}')


In [ ]:
# 6.5.3 -- PCA visualisation of clusters
#
# PCA compresses our N-dimensional feature space into 2 dimensions
# so we can plot the clusters on a scatter chart.
# The two PCA axes explain as much variance as possible from the originals.

pca = PCA(n_components=2, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

explained = pca.explained_variance_ratio_
print(f'PCA variance explained:')
print(f'  PC1: {explained[0]*100:.1f}%')
print(f'  PC2: {explained[1]*100:.1f}%')
print(f'  Total: {sum(explained)*100:.1f}%')

fig, ax = plt.subplots(figsize=(10, 7))
colours = plt.cm.tab10.colors
for cluster_id in range(best_k):
    mask = labels == cluster_id
    ax.scatter(
        X_pca[mask, 0], X_pca[mask, 1],
        c=[colours[cluster_id]], alpha=0.35, s=12, linewidths=0,
        label=f'Cluster {cluster_id} (n={mask.sum():,})'
    )
# Plot cluster centres
centres_pca = pca.transform(km_final.cluster_centers_)
ax.scatter(centres_pca[:, 0], centres_pca[:, 1],
           c='black', s=180, marker='X', zorder=10, label='Cluster centres')
ax.set_xlabel(f'PC1 ({explained[0]*100:.1f}% variance)')
ax.set_ylabel(f'PC2 ({explained[1]*100:.1f}% variance)')
ax.set_title(f'KMeans Clustering (k={best_k}) Visualised with PCA\nSO 2025 Developer Archetypes')
ax.legend(fontsize=9, markerscale=2)
plt.tight_layout()
plt.show()


---

## Section 6.6 -- Handling Class Imbalance

Real-world classification datasets are rarely balanced. In the SO 2025 data,
Python users may outnumber non-Python users 3:1. A naive classifier that
always predicts the majority class gets high accuracy but is useless.

**Three strategies:**

**Class weights** -- tell the algorithm to penalise errors on the minority
class more heavily. Built into most scikit-learn classifiers via `class_weight='balanced'`.
No extra data needed, no risk of overfitting synthetic samples.

**SMOTE (Synthetic Minority Over-sampling Technique)** -- generates synthetic
minority-class samples by interpolating between existing ones.
Increases minority representation without simply duplicating rows.

**Threshold adjustment** -- instead of the default 0.5 decision threshold,
choose a threshold that optimises the metric you actually care about
(precision, recall, or F1).


In [ ]:
# 6.6.1 -- Diagnose and quantify class imbalance

import subprocess
subprocess.run(['pip', 'install', 'imbalanced-learn', '-q'], check=False)

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Build binary classification dataset
clf_feature_cols = [c for c in ['YearsCodePro', 'ConvertedCompYearly',
                                  'uses_sql', 'uses_js', 'uses_ai']
                    if c in df.columns]

X_clf = df[clf_feature_cols].copy()
for col in clf_feature_cols:
    med = X_clf[col].median()
    X_clf[col] = X_clf[col].fillna(med if pd.notna(med) else 0)
y_clf = df['uses_python']

X_tr, X_te, y_tr, y_te = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=RANDOM_STATE, stratify=y_clf
)

class_counts = y_tr.value_counts()
majority = class_counts.max()
minority = class_counts.min()
ratio    = majority / minority

print('Class distribution in training set:')
for cls, cnt in class_counts.items():
    pct = cnt / len(y_tr) * 100
    label = 'Python' if cls == 1 else 'Non-Python'
    print(f'  {label} ({cls}): {cnt:,}  ({pct:.1f}%)')
print(f'Imbalance ratio: {ratio:.2f}:1')
if ratio > 3:
    print('  -> Significant imbalance: class weights or SMOTE recommended')
elif ratio > 1.5:
    print('  -> Mild imbalance: class weights usually sufficient')
else:
    print('  -> Balanced: no special handling needed')


In [ ]:
# 6.6.2 -- Compare three approaches: baseline, class weights, SMOTE

preprocess = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler()),
])

X_tr_proc = preprocess.fit_transform(X_tr)
X_te_proc = preprocess.transform(X_te)

results_imb = {}

# 1. Baseline: no imbalance handling
lr_base = LogisticRegression(max_iter=500, random_state=RANDOM_STATE)
lr_base.fit(X_tr_proc, y_tr)
results_imb['Baseline'] = lr_base.predict(X_te_proc)

# 2. Class weights: penalise minority class errors more
lr_weighted = LogisticRegression(max_iter=500, class_weight='balanced',
                                   random_state=RANDOM_STATE)
lr_weighted.fit(X_tr_proc, y_tr)
results_imb['Class weights'] = lr_weighted.predict(X_te_proc)

# 3. SMOTE: synthesise minority class samples
smote = SMOTE(random_state=RANDOM_STATE)
X_tr_sm, y_tr_sm = smote.fit_resample(X_tr_proc, y_tr)
print(f'After SMOTE -- class 0: {(y_tr_sm==0).sum():,}  class 1: {(y_tr_sm==1).sum():,}')
lr_smote = LogisticRegression(max_iter=500, random_state=RANDOM_STATE)
lr_smote.fit(X_tr_sm, y_tr_sm)
results_imb['SMOTE'] = lr_smote.predict(X_te_proc)

# Compare
print()
for name, y_pred in results_imb.items():
    print(f'=== {name} ===')
    print(classification_report(y_te, y_pred,
                                 target_names=['Non-Python','Python'],
                                 zero_division=0))


In [ ]:
# 6.6.3 -- Threshold adjustment: precision-recall tradeoff
#
# The default threshold of 0.5 is rarely optimal.
# Plot the precision-recall curve and choose the threshold
# that maximises F1 for the minority class.

from sklearn.metrics import precision_recall_curve, f1_score

y_proba = lr_weighted.predict_proba(X_te_proc)[:, 1]  # P(Python=1)

precisions, recalls, thresholds = precision_recall_curve(y_te, y_proba)

# F1 at each threshold
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (
    precisions[:-1] + recalls[:-1] + 1e-9
)
best_thresh_idx = f1_scores.argmax()
best_thresh     = thresholds[best_thresh_idx]
best_f1         = f1_scores[best_thresh_idx]

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(thresholds, precisions[:-1], '#2E75B6', linewidth=2, label='Precision')
axes[0].plot(thresholds, recalls[:-1],    '#E8722A', linewidth=2, label='Recall')
axes[0].plot(thresholds, f1_scores,       'green',   linewidth=2, label='F1')
axes[0].axvline(best_thresh, color='black', linestyle='--', linewidth=1.5,
                label=f'Best threshold={best_thresh:.2f}')
axes[0].set_xlabel('Decision threshold')
axes[0].set_ylabel('Score')
axes[0].set_title('Precision, Recall, F1 vs Threshold')
axes[0].legend(fontsize=9)

axes[1].plot(recalls, precisions, '#2E75B6', linewidth=2)
axes[1].scatter(recalls[best_thresh_idx], precisions[best_thresh_idx],
                s=100, color='red', zorder=5,
                label=f'Best F1={best_f1:.3f} @ {best_thresh:.2f}')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend(fontsize=9)

plt.suptitle('Threshold Optimisation for Python Usage Classifier',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

# Apply best threshold
y_pred_tuned = (y_proba >= best_thresh).astype(int)
default_f1   = f1_score(y_te, results_imb['Class weights'], average='weighted')
tuned_f1     = f1_score(y_te, y_pred_tuned, average='weighted')
print(f'Default threshold (0.50) weighted F1: {default_f1:.4f}')
print(f'Optimised threshold ({best_thresh:.2f}) weighted F1: {tuned_f1:.4f}')


---

## Section 6.7 -- Probability Calibration

A classifier that outputs `predict_proba([...]) = 0.8` is claiming
80% confidence. A **calibrated** classifier means exactly that:
among all predictions with confidence ~0.8, roughly 80% are actually correct.

Many models are systematically miscalibrated:
- **Random Forests** tend to push probabilities toward 0.5 (under-confident)
- **SVMs and Naive Bayes** often produce extreme probabilities (over-confident)

The **reliability diagram** (calibration curve) visualises this.
The **Brier score** quantifies it: lower is better, 0 is perfect, 0.25 is a
coin-flip on a balanced binary problem.

Calibration matters whenever you use probabilities directly -- risk scoring,
decision thresholds, or ranking outputs by confidence.


In [ ]:
# 6.7.1 -- Calibration curves and Brier scores

from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Train three classifiers on the same data
models_cal = {
    'Logistic Regression': LogisticRegression(max_iter=500, random_state=RANDOM_STATE),
    'Random Forest':       RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'Gradient Boosting':   GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),
}

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Perfect calibration reference line
axes[0].plot([0,1],[0,1], 'k--', linewidth=1.5, label='Perfectly calibrated')

colours = ['#2E75B6', '#E8722A', 'green']
brier_scores = {}

for (name, model), colour in zip(models_cal.items(), colours):
    model.fit(X_tr_proc, y_tr)
    proba = model.predict_proba(X_te_proc)[:, 1]
    brier = brier_score_loss(y_te, proba)
    brier_scores[name] = brier
    CalibrationDisplay.from_predictions(
        y_te, proba, n_bins=10, ax=axes[0],
        name=f'{name} (Brier={brier:.3f})', color=colour
    )

axes[0].set_title('Calibration Curves\n(closer to diagonal = better calibrated)')
axes[0].legend(fontsize=8)

# Calibrate Random Forest with isotonic regression
rf_uncal  = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf_cal    = CalibratedClassifierCV(rf_uncal, method='isotonic', cv=5)
rf_cal.fit(X_tr_proc, y_tr)
proba_cal = rf_cal.predict_proba(X_te_proc)[:, 1]
brier_cal = brier_score_loss(y_te, proba_cal)

rf_uncal2 = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf_uncal2.fit(X_tr_proc, y_tr)
proba_uncal = rf_uncal2.predict_proba(X_te_proc)[:, 1]
brier_uncal = brier_score_loss(y_te, proba_uncal)

axes[1].plot([0,1],[0,1],'k--',linewidth=1.5, label='Perfectly calibrated')
CalibrationDisplay.from_predictions(
    y_te, proba_uncal, n_bins=10, ax=axes[1],
    name=f'RF uncalibrated (Brier={brier_uncal:.3f})', color='#E8722A'
)
CalibrationDisplay.from_predictions(
    y_te, proba_cal, n_bins=10, ax=axes[1],
    name=f'RF + isotonic calibration (Brier={brier_cal:.3f})', color='#2E75B6'
)
axes[1].set_title('Effect of Isotonic Calibration on Random Forest')
axes[1].legend(fontsize=9)

plt.suptitle('Probability Calibration: SO 2025 Python Usage Classifier',
             fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print('Brier scores (lower = better, 0.25 = coin flip):')
for name, score in brier_scores.items():
    print(f'  {name:<25} {score:.4f}')
print(f'  RF + calibration        {brier_cal:.4f}  (vs uncalibrated {brier_uncal:.4f})')


---

## Section 6.8 -- FeatureUnion: Combining Feature Sets

`ColumnTransformer` applies different transformers to different columns
and concatenates the results. `FeatureUnion` does something complementary:
it applies multiple transformers to the **same** input and concatenates their outputs.

This is useful when you want to extract several different feature representations
from the same data -- for example, combining raw scaled features with
polynomial features and feature interactions in a single pipeline step.

Together, `ColumnTransformer` + `FeatureUnion` + `Pipeline` give you complete
control over feature engineering without any data leakage.


In [ ]:
# 6.8.1 -- FeatureUnion: combine raw features with polynomial interactions

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

# FeatureUnion applies each transformer to the full input
# and concatenates the results horizontally
feature_union = FeatureUnion([
    # Branch 1: original scaled features
    ('scaled', StandardScaler()),
    # Branch 2: degree-2 polynomial features (captures interactions)
    # e.g. YearsCodePro * uses_python
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
])

# Wrap in a full pipeline with a classifier
union_pipeline = ImbPipeline([
    ('imputer',  SimpleImputer(strategy='median')),
    ('features', feature_union),
    ('pca',      PCA(n_components=0.95)),  # keep 95% of variance
    ('clf',      LogisticRegression(max_iter=500, class_weight='balanced',
                                     random_state=RANDOM_STATE)),
])

union_pipeline.fit(X_tr, y_tr)
y_pred_union = union_pipeline.predict(X_te)

# Show how many features each stage produces
imp_out   = SimpleImputer().fit_transform(X_tr)
union_out = feature_union.fit_transform(imp_out)
pca_temp  = PCA(n_components=0.95).fit(union_out)

print('Feature pipeline stages:')
print(f'  Input features:              {X_tr.shape[1]}')
print(f'  After FeatureUnion:')
print(f'    Branch 1 (scaled):         {X_tr.shape[1]}')
n_poly = PolynomialFeatures(degree=2, include_bias=False).fit(imp_out).n_output_features_
print(f'    Branch 2 (polynomial):     {n_poly}')
print(f'    Combined:                  {union_out.shape[1]}')
print(f'  After PCA (95% variance):    {pca_temp.n_components_}')
print()

from sklearn.metrics import f1_score as f1
base_f1  = f1(y_te, results_imb['Class weights'], average='weighted')
union_f1 = f1(y_te, y_pred_union, average='weighted')
print(f'Logistic Regression (raw features):         F1={base_f1:.4f}')
print(f'Logistic Regression (FeatureUnion + PCA):   F1={union_f1:.4f}')


---

## Chapter 6 Summary

Chapter 6 is the core ML chapter. Every concept here recurs in Chapter 7
(deep learning) -- the difference is that neural networks learn their own
features rather than using the hand-engineered ones we built here.

### Key Takeaways

- **The scikit-learn API contract:** `fit()`, `transform()`, `predict()`, `score()`.
  Every estimator and transformer follows it.
- **Pipeline** prevents data leakage and makes the full workflow serialisable.
- **ColumnTransformer** applies different preprocessing to numeric vs categorical columns.
- **Cross-validation** gives honest performance estimates. Never evaluate on training data.
- **Class imbalance** distorts accuracy. Use `class_weight='balanced'`, SMOTE,
  or threshold adjustment depending on the severity and your metric priorities.
- **Threshold optimisation** lets you trade precision for recall explicitly.
  The precision-recall curve shows the full tradeoff; pick the threshold that
  maximises your actual business metric.
- **Calibration** ensures predicted probabilities are trustworthy.
  `CalibratedClassifierCV` with isotonic regression fixes Random Forest miscalibration.
  The Brier score quantifies calibration quality.
- **FeatureUnion** combines multiple feature representations from the same input.
  Pair it with `ColumnTransformer` and `Pipeline` for complete feature engineering control.
- **Learning curves** diagnose bias vs variance -- the most actionable diagnostic.
- **RandomizedSearchCV** is more efficient than GridSearch for large hyperparameter spaces.

### Project Thread Status

| Task | Model | Key Result |
|------|-------|------------|
| Salary regression | Random Forest | Reported R^2, MAE |
| Python usage classification | Best of 4 models | Accuracy + confusion matrix |
| Hyperparameter tuning | RandomizedSearchCV | Improved accuracy |
| Developer clustering | KMeans + PCA | Archetypes identified |
| Class imbalance | SMOTE + class weights | Per-class F1 comparison |
| Threshold optimisation | Precision-recall curve | Best F1 threshold found |
| Calibration | Isotonic CalibratedClassifierCV | Brier score improved |
| Feature engineering | FeatureUnion + polynomial | Feature count comparison |

---

### What's Next: Chapter 7 -- Deep Learning with PyTorch

Chapter 7 introduces neural networks. You will build a salary regression MLP
and a developer role classifier from scratch in PyTorch, understanding every
component: tensors, layers, activation functions, loss functions, and the
training loop. The scikit-learn patterns from this chapter appear again --
now implemented manually so you understand what the framework does for you.

---

*End of Chapter 6 -- Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
